In [63]:
#########################################
##The model deals with the prediction  ##
##predict required commodities for     ##
##the next year in general after taking##
## in the amounts of the other 5 years ##
#########################################

In [64]:
import pandas as pd
import math
import numpy as np
from prophet import Prophet
df  = pd.read_csv('data.csv')

In [65]:
#per county and loop for all
#rename the columns
new_names  = {"Baringo County": 'baringo', "Bomet County": 'bomet', "Bungoma County": 'bungoma', "Busia County": 'busia', "Elgeyo Marakwet County": 'elgeyo',
               "Embu County": 'embu', "Garissa County": 'garissa', "Homa Bay County": 'homabay', "Isiolo County": 'isiolo', "Kajiado County": 'kajiado',
                "Kakamega County": 'kakamega', "Kericho County": 'kericho', "Kiambu County": 'kiambu', "Kilifi County": 'kilifi', "Kirinyaga County": 'kirinyaga',
                "Kisii County": 'kisii', "Kisumu County": 'kisumu', "Kitui County": 'kitui', "Kwale County": 'kwale', "Laikipia County": 'laikipia', "Lamu County": 'lamu',
                "Machakos County": 'machakos', "Makueni County": 'makueni', "Mandera County": 'mandera', "Marsabit County": 'marsabit', "Meru County": 'meru', "Migori County": 'migori',
                "Mombasa County": 'mombasa', "Muranga County": 'muranga', "Nairobi County": 'nairobi', "Nakuru County": 'nakuru', "Nandi County": 'nandi', "Narok County": 'narok',
                "Nyamira County": 'nyamira', "Nyandarua County": 'nyandarua', "Nyeri County": 'nyeri', "Samburu County": 'samburu', "Siaya County": 'siaya', "Taita Taveta County": 'taita', 
                "Tana River County": 'tanariver', "Tharaka Nithi County": 'tharaka', "Trans Nzoia County": 'transzoia', "Turkana County": 'turkana', "Uasin Gishu County": 'uasingishu',
                "Vihiga County": 'vihiga', "Wajir County": 'wajir', "West Pokot County": 'westpokot'}

df.rename(columns=new_names, inplace=True)

years = [ periodid for periodid in df['periodid']]

In [66]:
#get data for each county and  whole year
###########################################################################################
## EACH COUNTY GETS TOTAL DATA FOR A PARTICULAR YEAR                                    ###  
##I.E TAKES A PARTICULAR YEAR AND COUNTY, GETS THE DATA BY LOOPING THROUGH EACH MONTH  ###
#########################################################################################
period_ids  =  [ period for period in df['periodid']]
years = set([str(year)[:4] for year in period_ids], )


In [67]:
###############################################################
#get total, mean and median for every column (each  year begin from 202201 to 201812)
#ie summation from jan to dec each year
#
###############################################################

county_yearly_vals = []
def total_items_per_column(period=None,county=None):
    
    county_yearly_vals.clear()
   
    if county == None:
        print("Please select a county for now!")
    else: 
        try:
            #select the column data("county")
            county_values = df[county]
            #print(period)
            column_data = county_values.loc[df['periodid'] == period].sum()
            
            return column_data

        except Exception as e:
            print("Wasn't supposed to happen!")

#total_items_per_row(202201)

In [68]:
###############################################################
#get total, mean and median for every row (each  month begin from 202201 to 201812)
#ie summation across january on 2021 or accross feb
#import numpy to get mean
#get data for each county and  whole year for rows
###############################################################

total_items_each_row = []

county_yearly_vals = []
def total_items_per_row(period=None,county=None):
    total_items_each_row.clear()
    
    try:
        row_data = df.loc[df['periodid'] == period].sum()
        
        for k,v in new_names.items():
                #print(k,row_data[v],period)
                total_items_each_row.append(row_data[v])
        
        #print(statistics.fmean(total_items_each_row))
        total_items_per_period = np.sum(total_items_each_row)
        mean_total_items = np.mean(total_items_each_row)
        median_total_items = np.median(total_items_each_row)

        return {"sum": total_items_per_period, "mean": math.floor(mean_total_items), "median": median_total_items}

    except Exception as e:
        print("Wasn't supposed to happen!")
#total_items_per_row(202201)

In [69]:

#get total sum, mean and median per year
#the dict takes in a specific year and the total
###############################################################
##
## MONTH : BASED ON YEARS AND SUMS ACROSS ALL COUNTIES                    
##I.E TAKES A PARTICULAR YEAR AND GETS THE DATA ACROSS AND SUMS ALL FOR THAT YEAR LOOPING THROUGH EACH
###############################################################
datetime = []
totals = []

def yearly_reports(year=None):
    
    #2022,2021,2020,2019,2018

    for year in sorted(years):
            
            #locate all rows with specific years
            total_mean = []
            total_sum = []
            total_median = []
            for period in period_ids:
                if str(period)[:4] == str(year):
                    values = total_items_per_row(period)
                    #print(values, period)
                    total_sum.append(values['sum'])
            #print(monthly_totals)
            sums = np.sum(total_sum)
            datetime.append(year)
            totals.append(f"{sums:.2f}")


yearly_reports()

In [71]:
# Create a DataFrame with the dates and data
new_df = pd.DataFrame({'ds': pd.to_datetime(datetime),
                   'y': totals})

# Convert the data to numeric
new_df['y'] = pd.to_numeric(new_df['y'])

# Create a Prophet model and fit the data
m = Prophet()
m.fit(new_df)

# Create a DataFrame with the dates you want to predict
future = m.make_future_dataframe(periods=365)

# Make the predictions for the future dates
forecast = m.predict(future)

# Get the predicted values for 2023
predictions = forecast[forecast['ds'].dt.year == 2023]['yhat'].values

# Print the predictions
print(f"{math.floor(predictions):.2f}")



17:45:54 - cmdstanpy - INFO - Chain [1] start processing
17:45:54 - cmdstanpy - INFO - Chain [1] done processing


5888714.00
